In [1]:
import pandas as pd
import numpy as np
import os
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

The reviews are stored in txt files individually and given in folders "train" and "test".The "train" folder contains two folders: "pos" and "neg". We have to read each file in train/pos and train/neg into the data frame. To achive this we first create a list of lists where each inner list contains one review and its label. Then we create our data frame from this list

In [2]:
filenames_pos = os.listdir("/Users/abdullahkarakus/Documents/Github/IMDB/aclImdb/train/pos")

filenames_neg = os.listdir("/Users/abdullahkarakus/Documents/Github/IMDB/aclImdb/train/neg")

review_list = []

for file in filenames_pos:
    with open("/Users/abdullahkarakus/Documents/Github/IMDB/aclImdb/train/pos/" + file, "r") as target_file:
        review_list.append([target_file.read(),1])
        

for file in filenames_neg:
    with open("/Users/abdullahkarakus/Documents/Github/IMDB/aclImdb/train/neg/" + file, "r") as target_file:
        review_list.append([target_file.read(),0])

training_data = pd.DataFrame(data = review_list, columns = ["reviews","labels"])

Let's check our data frame.

In [3]:
training_data

,reviews,labels
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1
...,...,...
24995,"My comments may be a bit of a spoiler, for wha...",0
24996,"The ""saucy"" misadventures of four au pairs who...",0
24997,"Oh, those Italians! Assuming that movies about...",0
24998,Eight academy nominations? It's beyond belief....,0


We do the same thing for the test data.

In [4]:
filenames_pos = os.listdir("/Users/abdullahkarakus/Documents/Github/IMDB/aclImdb/test/pos")

filenames_neg = os.listdir("/Users/abdullahkarakus/Documents/Github/IMDB/aclImdb/test/neg")

review_list = []

for file in filenames_pos:
    with open("/Users/abdullahkarakus/Documents/Github/IMDB/aclImdb/test/pos/" + file, "r") as target_file:
        review_list.append([target_file.read(),1])
        

for file in filenames_neg:
    with open("/Users/abdullahkarakus/Documents/Github/IMDB/aclImdb/test/neg/" + file, "r") as target_file:
        review_list.append([target_file.read(),0])

test_data = pd.DataFrame(data = review_list, columns = ["reviews","labels"])

In [5]:
test_data

,reviews,labels
0,"Based on an actual story, John Boorman shows t...",1
1,This is a gem. As a Film Four production - the...,1
2,"I really like this show. It has drama, romance...",1
3,This is the best 3-D experience Disney has at ...,1
4,"Of the Korean movies I've seen, only three had...",1
...,...,...
24995,With actors like Depardieu and Richard it is r...,0
24996,If you like to get a couple of fleeting glimps...,0
24997,When something can be anything you want it to ...,0
24998,"I had heard good things about ""States of Grace...",0


In [6]:
for review in training_data.sample(10)["reviews"]:
    print(review)

The characters were alive and interesting, the plot was excellently paced, the pyro effects were masterfully accomplished, and it takes a basic love triangle story and tosses in a science-fiction element into it. I could identify with many of the characters and their motivations made logical rational sense in the framework of the story.<br /><br />The camera-work was great, the audio clear and accurate, the background music perfectly chosen for effect, the singing firemen a nice talented memorable oddity, the sets brilliantly crafted, and the special effects performed with a skilled talent.<br /><br />I am a tad puzzled how an entire mini-carnival in a chain-store's parking lot could be powered by one single lamppost outlet. That seems impossible to say the least. The fight between the brothers near the end of the movie was brilliant though. Having Jim Varney in a non-clown role was a wonderful touch too as played the semi-serious role of a carny very well.
Much like the comedy duo of 

In [7]:
dataset = [training_data, test_data]

for data in dataset:
    data["reviews"] = data["reviews"].apply(lambda x: x.lower())
    data["reviews"] = data["reviews"].apply(lambda x: re.sub("<.*>", " ",x))
    data["reviews"] = data["reviews"].apply(lambda x: re.sub("('|\")","",x))
    data["reviews"] = data["reviews"].apply(lambda x: re.sub("[^A-Za-z]"," ",x))
    data["reviews"] = data["reviews"].apply(lambda x: re.sub("(?<=\s)[a-z](?=\s)","",x))

In [8]:
for review in training_data.sample(10)["reviews"]:
    print(review)

the film my name is modesty is based around an episode that takes up about one page in the   th modesty blaise novel called night of the morningstar  it describes an incident in which the young modesty     in the book  mid twenties in the film asserts her leadership in  war over  casino  as this is set before the actual blaise adventures her trusted sidekick willi garvin is not in the film  that is one of the main problems as the relationship between blaise and garvin was certainly always one of the fascinating aspects of the novels and the long running comic strip  the other problem is that the film is quite simply incredibly boring because it really is just one small episode blown up into  screenplay  the casting is okay but alexandra staden is not really convincing as the heroine and actually too old for the role to play the young modesty   get the impression that this film was  quick and dirty solution as not to lose the rights to the blaise franchise 
scarface has  major cult foll

In [9]:
tokenizer = Tokenizer(num_words = 2500)
tokenizer.fit_on_texts(training_data["reviews"])

In [10]:
tokenizer

In [11]:
X_train = tokenizer.texts_to_sequences(training_data["reviews"])
X_train = pad_sequences(X_train,maxlen = 800)
X_test = tokenizer.texts_to_sequences(test_data["reviews"])
X_test = pad_sequences(X_test,maxlen = 800)

In [12]:
model = Sequential()
model.add(layers.Embedding(2500,64))
model.add(layers.LSTM(16,dropout=0.4))
model.add(layers.Dense(1,activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", opt = "adam", metrics = ['accuracy'])

2021-11-21 21:11:05.828781: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-21 21:11:05.828972: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [13]:
Y_train = np.array(training_data["labels"])

model.fit(X_train, Y_train, batch_size = 100, epochs = 3)

Train on 25000 samples
Epoch 1/3


2021-11-21 21:11:07.980515: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference___backward_cudnn_lstm_with_fallback_2609_2789' and '__inference___backward_standard_lstm_2910_3381_specialized_for_StatefulPartitionedCall_at___inference_distributed_function_3564' both implement 'lstm_83fb5858-269b-4dea-b452-9c05f2833e07' but their signatures do not match.


25000/25000 [==============================] - 526s 21ms/sample - loss: 0.5045 - accuracy: 0.7630
Epoch 2/3
25000/25000 [==============================] - 512s 20ms/sample - loss: 0.3695 - accuracy: 0.8420
Epoch 3/3
25000/25000 [==============================] - 505s 20ms/sample - loss: 0.3380 - accuracy: 0.8563


In [18]:
Y_estimate = model.predict(X_test)

2021-11-21 21:43:57.024181: W tensorflow/core/grappler/optimizers/implementation_selector.cc:310] Skipping optimization due to error while loading function libraries: Invalid argument: Functions '__inference_cudnn_lstm_with_fallback_6175' and '__inference_standard_lstm_6064_specialized_for_sequential_lstm_StatefulPartitionedCall_at___inference_distributed_function_6367' both implement 'lstm_47ae0eec-d1cf-4550-aac7-23242b09af09' but their signatures do not match.


In [19]:
Y_estimate

array([[0.6194659 ],
       [0.7488825 ],
       [0.9365021 ],
       ...,
       [0.05574858],
       [0.30517158],
       [0.08185074]], dtype=float32)

In [24]:
Y_test = np.array(test_data["labels"])
loss,acc = model.evaluate(X_test,Y_test,verbose = 0)

In [25]:
loss

0.35935144062042235

In [26]:
acc

0.84364